Part Two

Scrape the content of https://www.lemonde.fr/ and save it as a CSV.

We want: titles, subhead, article URL, whether it's premium or not, byline, article type, image URL.

Bonus, if you want to get fancy:

Make the CSV file auto-updating. Use this tutorial (videoLinks to an external site., textLinks to an external site.) but just ignore the visualization/datawrapper aspect
Tips:

You're going to be missing some article URLs - you should use try/except on them to avoid them being a problem, buuuut after that I would recommend printing the element to see why your approach isn't working (it's 100% possible to get all of the URLs!)
If you want to make sure the element you're getting has a specific attribute, you can do something like .select_one("a[href]"). That will only give you anchor tags (links) that have hrefs.
Instead of yes/no for the premium question, you can think of it as "put text in this column if the article is premium, don't put text in it if it is not"

In [217]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import pandas as pd

In [190]:
url = "https://www.lemonde.fr/en/"
r = requests.get(url)

In [191]:
doc = BeautifulSoup(r.text)

In [192]:
print(doc)

<!DOCTYPE html>
<html lang="en" prefix="og: https://ogp.me/ns#"> <head> <meta charset="utf-8"/> <meta content="IE=edge" http-equiv="X-UA-Compatible"/> <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/> <meta content="no-referrer-when-downgrade" name="referrer"/> <meta content="#ffffff" name="theme-color"/> <script async="1" data-gdpr-purposes="personalization" data-gdpr-src="https://www.lemonde.fr/en/bucket/resources/english/js/chartbeatMab.88d00fd6428d5de2.js" type="text/plain"></script> <link data-rh="true" href="https://www.lemonde.fr/en/" hreflang="en-US" rel="alternate"/> <link data-rh="true" href="https://www.lemonde.fr/en/" hreflang="en" rel="alternate"/> <link data-rh="true" href="https://www.lemonde.fr/en/" hreflang="en-CA" rel="alternate"/> <link data-rh="true" href="https://www.lemonde.fr/en/" hreflang="en-GB" rel="alternate"/> <link href="//img.lemde.fr" rel="preconnect"/> <link as="image" fetchpriority="high" imagesizes="100vw" imagesrc

In [199]:
articles = doc.find_all("div", class_="article")
articles

[<div class="article article--main"> <div class="lmd-link-clickarea"> <span class="icon__premium icon--outside" id="zone1-premium-81909"><span class="sr-only">Subscribers only</span></span><h1 class="article__title"><a aria-describedby="zone1-premium-81909" class="lmd-link-clickarea__link" href="https://www.lemonde.fr/en/international/article/2025/06/18/behind-trump-and-macron-s-war-of-words-g7-reveals-deep-western-divisions_6742454_4.html"> <p class="article__title-label">Behind Trump and Macron's war of words, G7 reveals deep Western divisions</p> </a> </h1> <div class="article__media-container"> <picture class="article__media"> <img alt="Emmanuel Macron, Mark Carney and Donald Trump at the G7 summit in Kananaskis, Canada, on June 16, 2025." height="2" loading="eager" sizes="(min-width: 1024px) 421px, 100vw" src="https://img.lemde.fr/2025/06/17/0/0/5899/3933/400/266/75/0/dc3b40a_upload-1-mbf6jrce2xan-jmug250616099.jpg" srcset=" https://img.lemde.fr/2025/06/17/0/0/5899/3933/800/533/75

In [201]:
links = [article.find("a").get("href") for article in articles]
links

['https://www.lemonde.fr/en/international/article/2025/06/18/behind-trump-and-macron-s-war-of-words-g7-reveals-deep-western-divisions_6742454_4.html',
 'https://www.lemonde.fr/en/international/article/2025/06/18/iran-says-it-fired-hypersonic-missiles-at-israel-after-israel-strikes-tehran_6742452_4.html',
 'https://www.lemonde.fr/en/international/article/2025/06/17/china-iran-s-main-economic-backer-remains-a-bystander-to-the-middle-east-conflict_6742440_4.html',
 'https://www.lemonde.fr/en/politics/article/2025/06/18/six-months-later-bayrou-is-still-france-s-pm-and-not-finished_6742448_5.html',
 'https://www.lemonde.fr/en/pixels/article/2025/06/18/on-x-ads-are-based-on-political-criteria-a-prohibited-practice_6742451_13.html',
 'https://www.lemonde.fr/en/lifestyle/article/2025/06/18/is-it-ever-a-good-idea-to-wear-your-clothes-backwards_6742443_37.html',
 'https://www.lemonde.fr/en/opinion/article/2025/06/18/iran-israel-preventive-war-and-regime-change-are-rarely-met-with-success_6742460

In [ ]:
#trying code for extracting data from first link

links[0]
r = requests.get(links[0])
doc = BeautifulSoup(r.text, "html.parser")
print(doc.find("h1").text.strip())
print(doc.find("p", class_="article__desc").text.strip())
print(doc.find("p", class_="article__status").text.strip())
print(doc.find("p", class_="meta meta__publisher meta__publisher--header").text.strip())
print(doc.find("ul", class_="breadcrumb").text.strip())
print(doc.select_one("#habillagepub > section > section > article > figure > picture > img")['src'])

Behind Trump and Macron's war of words, G7 reveals deep Western divisions
From Iran to Ukraine to trade disputes, tensions have escalated between the American president and his fellow G7 leaders. Donald Trump said on Tuesday that Emmanuel Macron 'always gets it wrong' and is 'publicity seeking.'
Subscribers only
By Philippe Ricard
World      Diplomacy
https://img.lemde.fr/2025/06/18/0/0/5900/3933/664/0/75/0/5ec6b8e_upload-1-yr5hstvhnqbd-jmug250617055.jpg


In [228]:
data_list= []
for url in links:
    r = requests.get(url)
    doc = BeautifulSoup(r.text, "html.parser")
    data_dict = {}

    data_dict["article_link"] = url
    data_dict["title"] = doc.find("h1").text.strip()
    data_dict["subhead"] = doc.find("p", class_="article__desc").text.strip()
    try:
        data_dict["status"] = doc.find("p", class_="article__status").text.strip()
    except:
        data_dict["status"] = "not premium"
    try:
        data_dict["byline"] = doc.find("p", class_="meta meta__publisher meta__publisher--header").text.strip()
    except:
        data_dict["byline"] = "no author name"
    data_dict["article_type"] = doc.find("ul", class_="breadcrumb").text.strip()
    try:
        data_dict["image_url"] = doc.select_one("#habillagepub > section > section > article > figure > picture > img")['src']
    except:
        data_dict["image_url"] = "no image"
    data_list.append(data_dict)


In [231]:
df = pd.DataFrame(data_list)
df

,article_link,title,subhead,status,byline,article_type,image_url
0,https://www.lemonde.fr/en/international/articl...,"Behind Trump and Macron's war of words, G7 rev...","From Iran to Ukraine to trade disputes, tensio...",Subscribers only,By Philippe Ricard,World Diplomacy,https://img.lemde.fr/2025/06/18/0/0/5900/3933/...
1,https://www.lemonde.fr/en/international/articl...,Iran says it fired hypersonic missiles at Isra...,As the conflict between Iran and Israel enters...,not premium,Le Monde with AFP,World Middle East wars,https://img.lemde.fr/2025/06/17/0/0/6500/4333/...
2,https://www.lemonde.fr/en/international/articl...,"China, Iran's main economic backer, remains a ...",As Chinese diplomacy becomes increasingly acti...,Subscribers only,By Harold Thibault (Beijing (China) correspond...,World China,https://img.lemde.fr/2025/06/17/0/0/3997/2665/...
3,https://www.lemonde.fr/en/politics/article/202...,"Six months later, Bayrou is still France's PM ...",With no majority to back him in the Assemblée ...,Subscribers only,By Nathalie Segaunes,French Politics François Bayrou government,https://img.lemde.fr/2025/06/16/0/0/4674/7011/...
4,https://www.lemonde.fr/en/pixels/article/2025/...,X allowed ads to be targeted according to poli...,"The targeting method, which is banned in Europ...",Subscribers only,By Damien Leloup,Pixels X / Twitter,no image
5,https://www.lemonde.fr/en/lifestyle/article/20...,Is it ever a good idea to… wear your clothes b...,"In fashion, as in life, it is important to kno...",not premium,no author name,Lifestyle Is it ever a good idea to...,https://img.lemde.fr/2025/05/16/0/0/3543/4488/...
6,https://www.lemonde.fr/en/opinion/article/2025...,Iran-Israel: 'Preventive war and regime change...,"Fundamentally committed to international law, ...",Subscribers only,no author name,Opinion Israel-Iran conflict,no image
7,https://www.lemonde.fr/en/culture/article/2025...,Netflix and French television group TF1 strike...,"Starting summer 2026, French users will access...",not premium,Le Monde with AFP,Culture French Economy,no image
8,https://www.lemonde.fr/en/economy/article/2025...,"In Saint Petersburg, 'Putin's Davos' features ...","For three days from June 18, the Saint Petersb...",Subscribers only,"By Benjamin Quénelle (Moscow (Russia), special...",Economy Russia,https://img.lemde.fr/2025/06/17/0/0/3500/2326/...
9,https://www.lemonde.fr/en/economy/article/2025...,Takata airbags: Stop-drive order issued for Ci...,After a Citroën C3 driver died on June 11 when...,Subscribers only,By Sophie Fay (with AFP),Economy Automobile industry,https://img.lemde.fr/2025/06/17/0/0/8256/5504/...


In [ ]:
df.to_csv("scraped_lemonde.csv")